### People counting using YOLOv8

It is suggested to run this cose using Google Colab.

**NOTE**: remember to use GPU to run the code

In [ ]:
# Install YOLOv8
%pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
# Install Supervision
%pip install supervision==0.2.0

from IPython import display
display.clear_output()

import supervision as sv
print("supervision", sv.__version__)

In [ ]:
# Insert the path to the video to analyze
VIDEO_PATH = ""
video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)
print(video_info)

In [ ]:
from ultralytics import YOLO

# Import the YOLOv8 model pretrained on COCO dataset
model = YOLO('yolov8s.pt')

In [ ]:
import numpy as np
import time

In [ ]:
# Define a polygon zone (4 vertexes)
polygon = np.array([
    [1725, 1550],
    [2725, 1550],
    [3500, 2160],
    [1250, 2160]
])
zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

In [ ]:
# initiate annotators
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)

In [ ]:
count_people = []

In [ ]:
def process_frame(frame: np.ndarray, _) -> np.ndarray:
    # detect
    results = model(frame, imgsz=1280)[0]
    detections = sv.Detections.from_yolov8(results)
    # Filter the detection to people only and set min level of confidence
    detections = detections[detections.class_id == 0 & (detections.confidence > 0.6)]
    newrow = {
        "Timestamp": time.time(),
        "N_people": len(detections)
    }
    count_people.append(newrow)
    zone.trigger(detections=detections)

    # annotate
    box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
    labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, confidence, class_id, _ in detections]
    frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
    frame = zone_annotator.annotate(scene=frame)

    return frame

In [ ]:
# Process the whole video
sv.process_video(source_path=VIDEO_PATH, target_path=f"/content/video-result.mp4", callback=process_frame)

from IPython import display
display.clear_output()

In [ ]:
import pandas as pd
df = pd.DataFrame(count_people)
df.to_csv("/content/count_people.csv")